In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import skimage
from skimage.io import imshow, imread, imsave
import numpy as np
from glob import glob
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras as K

In [ ]:
#Setting variables to hold the input and output directory paths
in_dir = '/content/drive/My Drive/Original'
out_dir = '/content/drive/My Drive/Otsu_Blur'

In [ ]:
#Read the input image file names with paths into a list
in_files = in_dir + '/*.png' #all files with .png extension will be selected
img_names = glob(in_files) #glob returns a list of file names with paths that match the in_files pattern
#print(img_names) #prints the list of files names

In [ ]:
#Applying Otsu's method with blur
for fn in img_names: #fn = file name
    print('processing %s...' % fn) #print a message to show which file is being processed

    # read the input image as gray (0 specifies that the image should be loaded in grayscale)
    im_gray = cv2.imread(fn, 0)

    blur = cv2.GaussianBlur(im_gray,(5,5),0)


    # threshold it with your saved threshold
                                 #0 is input as the threshold but Otsu's method automatically determines an optimal threshold
                                      # 255 is the maximum value assigned to pixels above the threshold
                                          #binary threshold is inverted (foreground becomes black, background becomes white)
                                                                   #apply Otsu's threshold
                                                                                  #the function returns two values, we only want the second one ie the threshold imagae
    im_bw = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # write the result to disk in the previously created output directory
    name = os.path.basename(fn) #extract the basename of the file
    outfile = out_dir + '/' + name #creates output file path by combining output directory with basename
    cv2.imwrite(outfile, im_bw) #image is written(saved) to the file path set above

processing /content/drive/My Drive/Original/9acf303a-fcba-4e52-be3e-8d061aa5e72a.png...
processing /content/drive/My Drive/Original/9e793460-d94c-47ac-b3a7-0d6ad86357da.png...
processing /content/drive/My Drive/Original/7eb95580-693f-41b6-9338-174be1430c23.png...
processing /content/drive/My Drive/Original/25c6cfb1-01d0-4864-87f9-02b7593683c7.png...
processing /content/drive/My Drive/Original/9af1ee02-87fd-42bc-963e-fa9ac3eb45bb.png...
processing /content/drive/My Drive/Original/50d9fc90-8642-4bb6-91a7-0069a7e37392.png...
processing /content/drive/My Drive/Original/b9d65ef3-1adc-4479-abee-2e422f5a613a.png...
processing /content/drive/My Drive/Original/6d05ed67-43e2-43af-9d57-70c7a6eeef5e.png...
processing /content/drive/My Drive/Original/3768c577-4e5e-4d70-8c4d-93d8aacd5baa.png...
processing /content/drive/My Drive/Original/bdb5d571-27bc-4a42-ae45-aa5a9af40a81.png...
processing /content/drive/My Drive/Original/03cbfcf2-8da6-462c-b30d-16d63357371f.png...
processing /content/drive/My Dri

In [ ]:
out_files =  out_dir + '/*.png'
out_names = glob(out_files)


In [ ]:
dice_total=0.0
jacc_total=0.0
image_count=0

In [ ]:
#Calculating dice coefficient

def dice_coef2(a, b):

    hi = a/255 #dividing by 255 so that the values are in the range [0,1] -standardising
    pred = (255-b)/255

    hi= K.backend.flatten(hi) #flattening the pixel values into a 1D array

    pred= K.backend.flatten(pred)

    numerator= 2 * K.backend.sum(hi*pred)

    denominator= K.backend.sum(hi+pred)
    dice = numerator / denominator

    return dice


In [ ]:
img_names.sort()

out_names.sort()

In [ ]:
# Iterate through the original and segmented image pairs
for gt_img_name, seg_img_name in zip(img_names, out_names):
    # Ensure the file names match
    if os.path.basename(gt_img_name) == os.path.basename(seg_img_name):
        # Read the original and segmented images in grayscale
        ground_truth = cv2.imread(gt_img_name, 0)
        segmented_img = cv2.imread(seg_img_name, 0)

        # Check if the images were loaded correctly
        if ground_truth is None or segmented_img is None:
            print(f"Error loading images: {gt_img_name} or {seg_img_name}")
            continue

        # Calculate Dice Coefficient for the current pair of images
        dice_score = dice_coef2(ground_truth, segmented_img)
        jacc_score = dice_score / (2-dice_score)
        # Convert TensorFlow tensor to a numerical value (using numpy)
        dice_value = dice_score.numpy()
        jacc_value = jacc_score.numpy()
        # Accumulate the Dice score
        dice_total += dice_value
        jacc_total += jacc_value
        image_count += 1

        # Print the Dice Coefficient for the current image
        print(f"Dice Coefficient for {os.path.basename(gt_img_name)}: {dice_value}")
    else:
        print(f"File mismatch: {os.path.basename(gt_img_name)} and {os.path.basename(seg_img_name)}")

# Calculate and print the average Dice score
if image_count > 0:
    average_dice_score = dice_total / image_count
    average_jacc_score = jacc_total / image_count
    print(f"\nAverage Dice Coefficient for the entire dataset: {average_dice_score}")
    print(f"\nAverage Jaccard Coefficient for the entire dataset: {average_jacc_score}")
else:
    print("No matching images found.")

Dice Coefficient for 002f20ad-2ace-499c-9335-c9080bc3e6b5.png: 0.9091416766594598
Dice Coefficient for 003a89b0-a095-417a-8dd6-f408339bbc68.png: 0.8748423443992889
Dice Coefficient for 003d1cdc-0aec-430b-90e0-f4d82135b1ac.png: 0.9274812801833071
Dice Coefficient for 005e60b6-77b8-458c-b57c-bfe0c7e7df78.png: 0.8683684575763935
Dice Coefficient for 00a02700-2ea2-4590-9e15-ffc9160fd3de.png: 0.8965417997329027
Dice Coefficient for 00c8364b-8c85-4502-bcfe-64736fe76815.png: 0.8784060680414081
Dice Coefficient for 00d04a90-80e5-4bce-9511-1b64eabb7a47.png: 0.8394414405957155
Dice Coefficient for 0154dd8e-72f8-4d78-a4dd-93e139577bd1.png: 0.8537707494386011
Dice Coefficient for 01b38a56-13eb-447b-89ee-fc21cb40dcc1.png: 0.8490115947545778
Dice Coefficient for 01b99da8-b66b-464b-a7ee-1cadab8db18b.png: 0.8739133696995058
Dice Coefficient for 027ab174-e47a-44f8-b85b-372449eeb7d8.png: 0.9175479475706326
Dice Coefficient for 02ea7aa8-c142-4a3d-939f-e77689741341.png: 0.874748181568714
Dice Coefficient 

In [ ]:
len(out_names)

776

In [ ]:
len(img_names1)

776

In [ ]:
test_img_path = '/content/drive/My Drive/Original/5f66d20c-fbfe-44ec-a01b-8a09da24807e.png'
test_out_path= '/content/drive/My Drive/Original/test.png'

test_img= cv2.imread(test_img_path, 0)
cv2.imwrite(test_out_path, test_img)



True

In [ ]:
img= cv2.imread(test_out_path, 0)
otsuimg = cv2.imread('/content/drive/My Drive/Otsu_No_Blur/5f66d20c-fbfe-44ec-a01b-8a09da24807e.png', 0)

In [ ]:
print(img.shape)
print(otsuimg.shape)

(1200, 1600)
(1200, 1600)


In [ ]:
d= dice_coef2(img, otsuimg)
ds = d.numpy()
ds

0.19152882986111683